In [1]:
!pwd

/data5/galaxy/shell_dir/2018_3_17/jupyter_shell


#### proprocess data

In [2]:
import os
import sys
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import permutations
from multiprocessing import Pool

In [19]:
m6a_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/"
expre_dir = "/data5/galaxy/project/expression/DESeq2/"
names = get_genes_by_CpG()
# methylation level only have these tissue data.
subdir_list = ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
for sub_dir in subdir_list:
    print("##################################\n%s" % sub_dir)
#     df_m6a_up, df_m6a_common, df_m6a_down = class_m6a_by_diffPeak(os.path.join(m6a_dir, sub_dir))
#     df_m6a_up, df_m6a_down, df_m6a_common = filter_intersect(df_m6a_up, df_m6a_down, df_m6a_common)
#     df_m6a = pd.concat([df_m6a_up, df_m6a_common, df_m6a_down]).dropna()
    #
    df_gene_up, df_gene_down = class_gene_by_diffexpre(os.path.join(expre_dir, "%s.csv" % sub_dir))
#     df_gene_up = df_gene_up.merge(df_m6a, on="name", how="left").dropna()
#     df_gene_down = df_gene_down.merge(df_m6a, on="name", how="left").dropna()
    #
    df_methyl_up, df_methyl_down = foldchange_of_methylation(sub_dir)
    df_methyl_up = df_methyl_up[df_methyl_up["name"].isin(names)]
    df_methyl_down = df_methyl_down[df_methyl_down["name"].isin(names)]
#     df_up, df_down = merge_m6a_and_methyl(df_m6a, df_methyl_up, df_methyl_down)
###
    print("############ methylation ##########")
    statistic_proportion(df_methyl_up, df_methyl_down, df_gene_up, df_gene_down)
#     statistic_proportion(df_m6a_up, df_m6a_common, df_m6a_down, df_up, df_down)
# ###
#     print("########### methylation with expression ##############")
#     df_up = filter_each_row_by_expression(df_up, df_gene_up, df_gene_down)
#     df_down = filter_each_row_by_expression(df_down, df_gene_up, df_gene_down)
#     statistic_proportion(df_m6a_up, df_m6a_common, df_m6a_down, df_up, df_down)

##################################
heart_vs_brain
############ methylation ##########
methyl up
expre up	expre common	expre down
0.189427	0.607195	0.203377
methyl down
expre up	expre common	expre down
0.186441	0.624294	0.189266
##################################
kid_vs_brain
############ methylation ##########
methyl up
expre up	expre common	expre down
0.157438	0.689126	0.153436
methyl down
expre up	expre common	expre down
0.191882	0.626076	0.182042
##################################
lung_vs_brain
############ methylation ##########
methyl up
expre up	expre common	expre down
0.169188	0.655433	0.175378
methyl down
expre up	expre common	expre down
0.205621	0.582840	0.211538
##################################
heart_vs_kid
############ methylation ##########
methyl up
expre up	expre common	expre down
0.121573	0.736591	0.141836
methyl down
expre up	expre common	expre down
0.124859	0.778403	0.096738
##################################
lung_vs_heart
############ methylation ##########
methyl u

In [14]:
# def statistic_proportion(df_m6a_up, df_m6a_common, df_m6a_down, df_gene_up, df_gene_down):
#     print("up")
#     each_type_expre(df_gene_up)
#     print("down")
#     each_type_expre(df_gene_down)
    
# def each_type_expre(df):
#     total = len(df["name"].drop_duplicates())
#     m6a_up = len(df[df["name"].isin(df_m6a_up["name"])]["name"].drop_duplicates())
#     m6a_common = len(df[df["name"].isin(df_m6a_common["name"])]["name"].drop_duplicates())
#     m6a_down = len(df[df["name"].isin(df_m6a_down["name"])]["name"].drop_duplicates())
#     up_prop, common_prop, down_prop = (m6a_up / total), (m6a_common / total), (m6a_down/ total)
#     print("m6a up\tm6a common\tm6a down")
#     print("%f\t%f\t%f" % (up_prop, common_prop, down_prop))

In [16]:
def get_genes_by_CpG():
    cutoff = 0.35
    CpG_high = "/data5/galaxy/project/CpG_m6a_motif/fasta_seq/high_CpG.bed"
    for in_bed in [CpG_high]:
        df = pd.read_table(in_bed, sep="\t", header=None)
#         df_subset = df[df.iloc[:, 4] > cutoff]
        names = set(df.iloc[:, 3])
    return names

In [17]:
def statistic_proportion(df_methyl_up, df_methyl_down, df_gene_up, df_gene_down):
    print("methyl up")
    each_type_expre(df_methyl_up, df_gene_up, df_gene_down)
    print("methyl down")
    each_type_expre(df_methyl_down, df_gene_up, df_gene_down)
    
def each_type_expre(df, df_gene_up, df_gene_down):
    total = len(df["name"].drop_duplicates())
    up = len(df[df["name"].isin(df_gene_up["name"])]["name"].drop_duplicates())
    down = len(df[df["name"].isin(df_gene_down["name"])]["name"].drop_duplicates())
    common = total - up - down
    up_prop, common_prop, down_prop = (up / total), (common / total), (down/ total)
    print("expre up\texpre common\texpre down")
    print("%f\t%f\t%f" % (up_prop, common_prop, down_prop))

In [18]:
# 81--brain; 83--heart; 86--kid; 88--lung;
# ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
def foldchange_of_methylation(sub_dir):
    os.chdir("/data5/galaxy/project/methyl_m6a/data/roimethstat/")
    map_dict = {"brain": "E081", "heart": "E083", "kid": "E086", "lung": "E088"}
    tissue_1, tissue_2 = sub_dir.split("_")[0], sub_dir.split("_")[-1]
    try:
        num_1, num_2 = map_dict[tissue_1], map_dict[tissue_2]
    except KeyError:
        print("%s %s didn't have acoording DNA methylation data!" % (tissue_1, tissue_2))
    df_1 = pd.read_table("promoter_filtered_%s.bed" % num_1, header=None, names=["chr", "s", "e", "n", tissue_1, "strand"])
    # filtered_
    df_1["name"] = df_1["n"].str.split(":").str[0]
    df_2 = pd.read_table("promoter_filtered_%s.bed" % num_2, header=None, names=["chr", "s", "e", "n", tissue_2, "strand"])
    df_2["name"] = df_2["n"].str.split(":").str[0]
    df_methyl = df_1.merge(df_2, on="name").dropna()
    df_methyl["logFC"] = np.log2(df_methyl[tissue_1].astype(float) + 0.0001) - np.log2(df_methyl[tissue_2].astype(float) + 0.0001)
    df_methyl = df_methyl[["name", "logFC"]].sort_values(["logFC"], ascending=False)
#     print(df_methyl.head())
#     print(df_1[tissue_1].median(), df_2[tissue_2].median())
    df_methyl_up = df_methyl[df_methyl["logFC"] > 1]
#     df_methyl_common = df_methyl[ -1 < df_methyl["logFC"] < 1]
    df_methyl_down = df_methyl[df_methyl["logFC"] < -1]
    return df_methyl_up, df_methyl_down

In [12]:
# def class_and_merge(work_dir, df_methylation):
#     df_m6a_up, df_m6a_down = class_m6a_by_diffPeak(work_dir)
#     df_up, df_down, df_remain = merge_m6a_and_methyl(df_m6a_up, df_m6a_down, df_methylation)
#     return df_up, df_down, df_remain
    
# def class_m6a_by_diffPeak(work_dir):
#     os.chdir(work_dir)
#     m6a_up = pd.read_table("diff_peak_c3.0_cond1_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
#     m6a_common = pd.read_table("diff_peak_c3.0_common_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
#     m6a_down = pd.read_table("diff_peak_c3.0_cond2_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
#     # m6a score 计算方式
#     df_m6a_up = m6a_up.groupby(["name"]).mean().reset_index()
#     df_m6a_common = m6a_common.groupby(["name"]).mean().reset_index()
#     df_m6a_down = m6a_down.groupby(["name"]).mean().reset_index()
#     df_m6a_up, df_m6a_down, df_m6a_common = filter_intersect(df_m6a_up, df_m6a_down, df_m6a_common)
#     return df_m6a_up, df_m6a_common, df_m6a_down
    
# def filter_intersect(df_up, df_down, df_remain):
# #     print("before", len(df_up), len(df_down), len(df_remain))
#     up_genes, down_genes, remain_genes = set(df_up["name"]), set(df_down["name"]), set(df_remain["name"])
#     uniq_up = up_genes - down_genes - remain_genes
#     uniq_down = down_genes - up_genes - remain_genes
#     uniq_remain = remain_genes - up_genes - down_genes
#     df_up, df_down, df_remain = select_uniq(uniq_up, df_up), select_uniq(uniq_down, df_down), select_uniq(uniq_remain, df_remain)
# #     print("after", len(uniq_up), len(uniq_down), len(uniq_remain))
# #     print("after", len(df_up), len(df_down), len(df_remain))
#     return df_up, df_down, df_remain

# def merge_m6a_and_methyl(df_m6a, df_methyl_up, df_methyl_down):
#     df_up = df_m6a.merge(df_methyl_up, on="name", how="right").dropna().drop_duplicates()
# #     df_common = df_m6a.merge(df_methyl_common, on="name", how="right").dropna().drop_duplicates()
#     df_down = df_m6a.merge(df_methyl_down, on="name", how="right").dropna().drop_duplicates()
#     return df_up, df_down

In [7]:
def format_gene_name(df):
    df["name"] = df["Unnamed: 0"].str.split(".").str[0]
#     df = df.set_index(["name"])
    del df["Unnamed: 0"]
    return df

def class_gene_by_diffexpre(DESeq2_file):
    df = pd.read_csv(DESeq2_file)
    df = df[(abs(df.log2FoldChange) > 1) & (df.padj < 0.05)]
    df_up, df_down = df.copy()[df.log2FoldChange > 0], df.copy()[df.log2FoldChange < 0]
#     print(len(df_up), len(df_down))
    return format_gene_name(df_up), format_gene_name(df_down)

In [8]:
## 2.we find the Spearman rho between a row of expression data and each row of methylation data, and we keep the row of methylation data that produces the most-negative rho value
# def filter_each_row_by_expression(df, df_gene_up, df_gene_down):
# #     print(len(df))
#     df_pos, df_neg = df[df.logFC >0], df[df.logFC <0]
# #     print(len(df_pos), len(df_neg))
# #     print(df_pos.head())
# #     print(df_gene_up.head())
#     up_genes, down_genes, total_genes = df_gene_up.name, df_gene_down.name, list(set(df["name"].tolist()))
# #     remain_genes = [x for x in total_genes if (x not in up_genes) and (x not in down_genes)]
#     df_pos = df_pos[df_pos["name"].isin(down_genes)]
#     df_neg = df_neg[df_neg["name"].isin(up_genes)]
# #     print(len(df_pos), len(df_neg))
#     df_coordinate = pd.concat([df_pos, df_neg]).dropna()
# #     df_unchange = df[df["name"].isin(remain_genes)]
# #     print(len(df), len(df_coordinate), len(df_unchange))
#     return df_coordinate

In [9]:
# def intersect_m6a_expre_methyl(df_type, df_list, df_methyl):
#     df = df_list[0].merge(df_list[1], on="name", how="left").merge(df_methyl, on="name", how="left").dropna()
#     if df_type == "up":
#         df_coor = df[(df["logFC"] < 0) & (abs(df["logFC"]) > 1)][["name", "m6a_score", "log2FoldChange", "logFC"]]
#         df_unchange = df[(df["logFC"] < 0) & (abs(df["logFC"]) < 1)][["name", "m6a_score", "log2FoldChange", "logFC"]]
#     elif df_type == "down":
#         df_coor = df[(df["logFC"] > 0) & (abs(df["logFC"]) > 1)][["name", "m6a_score", "log2FoldChange", "logFC"]]
#         df_unchange = df[(df["logFC"] > 0) & (abs(df["logFC"]) < 1)][["name", "m6a_score", "log2FoldChange", "logFC"]]
#     else:
#         print("%s is not identified!" % df_type)
#     print(df.head(), len(df))
#     return df_coor, df_unchange
#     return df

In [10]:
def filter_intersect(df_up, df_down, df_remain):
#     print("before", len(df_up), len(df_down), len(df_remain))
    up_genes, down_genes, remain_genes = set(df_up["name"]), set(df_down["name"]), set(df_remain["name"])
    uniq_up = up_genes - down_genes - remain_genes
    uniq_down = down_genes - up_genes - remain_genes
    uniq_remain = remain_genes - up_genes - down_genes
    df_up, df_down, df_remain = select_uniq(uniq_up, df_up), select_uniq(uniq_down, df_down), select_uniq(uniq_remain, df_remain)
#     print("after", len(uniq_up), len(uniq_down), len(uniq_remain))
#     print("after", len(df_up), len(df_down), len(df_remain))
    return df_up, df_down, df_remain
    
def select_uniq(query_list, df):
    df = df[df["name"].isin(query_list)]
    return df

In [46]:
# def boxplot(df_up, df_down, df_remain, sub_dir):
# #     df_up_down = pd.DataFrame({"up": df_up_coor["logFC"], "up_unchanged": df_up_unchange["logFC"], 
# #                                "down": df_down_coor["logFC"], "down_unchanged": df_down_unchange["logFC"]})
# #     g = sns.boxplot(data=df_up_down, order=["down", "down_unchanged", "up", "up_unchanged"], saturation=0.5, showfliers=False)
#     df_up_down = pd.DataFrame({"up": df_up["logFC"], "down": df_down["logFC"], "remain": df_remain["logFC"]})
#     g = sns.boxplot(data=df_up_down, order=["down", "up", "remain"], saturation=0.5, showfliers=False)
#     g.set_ylabel("log2fc of methylation level", size = 14, color="black", alpha=1)
#     plt.title(sub_dir)
#     plt.savefig("/data5/galaxy/project/methyl_m6a/analysis_result_6/%s.pdf" % sub_dir)
#     plt.close()
#     print(len(df_up_down["down"].dropna()), len(df_up_down["up"].dropna()), len(df_up_down["remain"].dropna()))
#     print(df_up_down["down"].median(), df_up_down["up"].median(), df_up_down["remain"].median())
    # return df_up_down

In [2]:
# def prepare_plot_data(sub_dir, df_up, df_down, df_remain):
#     result_file = "/data5/galaxy/project/methyl_m6a/data/plot_data/%s.txt" % sub_dir
#     df_up["m6a_type"] = "up"
#     df_down["m6a_type"] = "down"
#     df_remain["m6a_type"] = "remain"
#     df = pd.concat([df_up, df_down, df_remain], sort=True)
#     df.to_csv(result_file, sep="\t", index=False)